In [12]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [13]:
rent_obj = []
all_property = []

for page in range(18,29):
    link = 'https://www.apartments.com/condos/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     page 1-7
#     results = soup.find_all('article', class_='premiumPlus placard')
#     page 7-28
    results = soup.find_all('article', class_='premium placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        try:
            lease_length = house_details.find('td', class_="leaseLength").text
        except:
            lease_length = 'null'
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Condos",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(1)
print(all_property)

[{'City_Rent': 'San Francisco', 'Type': 'Condos', 'State_Rent': 'CA', 'Zip_code': '94117', 'Bedroom_Info': [{'bedroom_type': '2 Beds', 'price': '$4,750'}], 'Full_Address': 'San Francisco, CA 94117', 'Rent_link': 'https://www.apartments.com/1377-fulton-st-san-francisco-ca/p60qvgv/', 'Bath_Info': '2 BAs', 'Square_Feet_Data': '1,400 Sq Ft', 'Square_Feet': '1,400', 'Lease_length': '12 Month Lease', 'Availability_check': 'Jul 10', '2 Beds': '$4,750'}, {'City_Rent': 'San Diego', 'Type': 'Condos', 'State_Rent': 'CA', 'Zip_code': '92101', 'Bedroom_Info': [{'bedroom_type': '2 Beds', 'price': '$3,100'}], 'Full_Address': 'San Diego, CA 92101', 'Rent_link': 'https://www.apartments.com/1225-island-ave-san-diego-ca-unit-101/9q1yfz5/', 'Bath_Info': '2 BAs', 'Square_Feet_Data': '1,988 Sq Ft', 'Square_Feet': '1,988', 'Lease_length': '6 Month Lease', 'Availability_check': 'Available Now', '2 Beds': '$3,100'}, {'City_Rent': 'Arcadia', 'Type': 'Condos', 'State_Rent': 'CA', 'Zip_code': '91007', 'Bedroom_In

In [14]:
rent_data = pd.DataFrame(all_property)
rent_data

,City_Rent,Type,State_Rent,Zip_code,Bedroom_Info,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Lease_length,Availability_check,2 Beds,1 Bed,Studio,3 Beds,4 Beds
0,San Francisco,Condos,CA,94117,"[{'bedroom_type': '2 Beds', 'price': '$4,750'}]","San Francisco, CA 94117",https://www.apartments.com/1377-fulton-st-san-...,2 BAs,"1,400 Sq Ft","1,400",12 Month Lease,Jul 10,"$4,750",NaN,NaN,NaN,NaN
1,San Diego,Condos,CA,92101,"[{'bedroom_type': '2 Beds', 'price': '$3,100'}]","San Diego, CA 92101",https://www.apartments.com/1225-island-ave-san...,2 BAs,"1,988 Sq Ft","1,988",6 Month Lease,Available Now,"$3,100",NaN,NaN,NaN,NaN
2,Arcadia,Condos,CA,91007,"[{'bedroom_type': '2 Beds', 'price': '$2,100'}]","Arcadia, CA 91007",https://www.apartments.com/1010-s-golden-w-ave...,1 BA,"1,200 Sq Ft","1,200",12 Month Lease,Available Now,"$2,100",NaN,NaN,NaN,NaN
3,South Lake Tahoe,Condos,CA,96150,"[{'bedroom_type': '2 Beds', 'price': '$1,700'}]","South Lake Tahoe, CA 96150",https://www.apartments.com/1223-bonanza-ave-so...,1 BA,850 Sq Ft,850,3 Month Lease,Available Now,"$1,700",NaN,NaN,NaN,NaN
4,Laguna Woods,Condos,CA,92637,"[{'bedroom_type': '1 Bed', 'price': '$2,795'}]","Laguna Woods, CA 92637",https://www.apartments.com/24055-paseo-del-lag...,2 BAs,975 Sq Ft,975,12 Month Lease,Available Now,NaN,"$2,795",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Torrance,Condos,CA,90502,"[{'bedroom_type': '1 Bed', 'price': '$1,875'}]","838 Coriander Dr, Torrance, CA 90502",https://www.apartments.com/838-coriander-dr-to...,1 BA,700 Sq Ft,700,12 Month Lease,Available Now,NaN,"$1,875",NaN,NaN,NaN
271,San Carlos,Condos,CA,94070,"[{'bedroom_type': '1 Bed', 'price': '$2,250'}]","San Carlos, CA 94070",https://www.apartments.com/515-chestnut-st-san...,1 BA,550 Sq Ft,550,12 Month Lease,Available Now,NaN,"$2,250",NaN,NaN,NaN
272,Santa Rosa,Condos,CA,95409,"[{'bedroom_type': '1 Bed', 'price': '$1,650'}]","Santa Rosa, CA 95409",https://www.apartments.com/6600-montecito-blvd...,1 BA,750 Sq Ft,750,6 Month Lease,Available Now,NaN,"$1,650",NaN,NaN,NaN
273,Santa Rosa,Condos,CA,95409,"[{'bedroom_type': '1 Bed', 'price': '$1,650'}]","Santa Rosa, CA 95409",https://www.apartments.com/6600-montecito-blvd...,1 BA,750 Sq Ft,750,6 Month Lease,Available Now,NaN,"$1,650",NaN,NaN,NaN


In [15]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q2-2020_Apartment.com_Condos_Data_page18-28.csv", index=False, header=True)